In [1]:
from pynq import Overlay
import asyncio
from psutil import cpu_percent

ol = Overlay("fact_intrpt.bit") #assumes you have the overlay bitstream in the current directory

In [2]:
ol.ip_dict

{'axi_intc': {'addr_range': 65536,
  'driver': pynq.overlay.DefaultIP,
  'fullpath': 'axi_intc',
  'gpio': {},
  'interrupts': {},
  'phys_addr': 1098907648,
  'state': None,
  'type': 'xilinx.com:ip:axi_intc:4.1'},
 'fact_intrpt': {'addr_range': 65536,
  'driver': pynq.overlay.DefaultIP,
  'fullpath': 'fact_intrpt',
  'gpio': {},
  'interrupts': {'interrupt': {'controller': 'axi_intc',
    'fullpath': 'fact_intrpt/interrupt',
    'index': 0}},
  'phys_addr': 1136656384,
  'state': None,
  'type': 'xilinx.com:hls:fact_intrpt:1.0'}}

# IP address map
You need to set up the interrupt controller in Vivado and get this information from there.

In [3]:
# IP's addresses
IP_CTRL = 0x00
AP_START = 0x1
GIER = 0x04
IP_IER = 0x08
IP_ISR = 0x0C
INTRPT_AP_DONE = 0x1
INTRPT_AP_READY = 0x2
OUT_REG = 0x10
INP_REG = 0x18

_INTRPT = INTRPT_AP_DONE 

fact_ip = ol.fact_intrpt
fact_ip.write(GIER, 0x1)
fact_ip.write(IP_IER, 0x3) #fix me

In [3]:
import asyncio
import time  
from datetime import datetime
import math

# You may want to comment out print statements in the read_ip and write_wait coroutines

# Coroutine that waits for an IP to be done.
async def service_interrupts(ip):
    factorial_input = 1
    fact_ip.write(INP_REG, factorial_input)
    fact_ip.write(IP_CTRL, AP_START)
    factorial_input += 1
#     while True: # uncomment me to run forever
    for i in range(200): # uncomment me to run for a fixe number of interrupts
        # Wait for the IP to finish.
        await ip.interrupt.wait()
#         print('Interrupt received. Status register: {}'.format(ip.read(IP_ISR)))
        if (ip.read(IP_ISR) & INTRPT_AP_DONE): # got a done interrupt
            ip.write(IP_ISR, INTRPT_AP_DONE) # clear done interrupt register
            print('interrupt received AP_DONE. The result is {}'.format(ip.read(OUT_REG))) # print out result
        elif (ip.read(IP_ISR) & INTRPT_AP_READY): # got a ready interrupt
            print('interrupt received AP_READY. Calling fact({})'.format(factorial_input))
            ip.write(IP_ISR, INTRPT_AP_READY) # clear the ready interrupt
            fact_ip.write(INP_REG, factorial_input) # write the input
            fact_ip.write(IP_CTRL, AP_START) # start the PL fact IP running
            factorial_input += 1
            

# Make a task from the read_ip coroutine and add it to the event loop
# ip_task = asyncio.ensure_future(service_interrupts(fact_ip))     

# Using psutil to record CPU utilization.
cpu_percent(percpu=True)  # Initializing the CPU monitoring.
asyncio.get_event_loop().run_until_complete(service_interrupts(fact_ip))
cpu_used = cpu_percent(percpu=True)

# Printing the CPU utilization
print('CPU Utilization = {cpu_used}'.format(**locals()))

# Removing the IP task from the event loop.
ip_task.cancel()

interrupt received AP_DONE. The result is 1
interrupt received AP_READY. Calling fact(2)
interrupt received AP_DONE. The result is 2
interrupt received AP_READY. Calling fact(3)
interrupt received AP_DONE. The result is 6
interrupt received AP_READY. Calling fact(4)
interrupt received AP_DONE. The result is 24
interrupt received AP_READY. Calling fact(5)
interrupt received AP_DONE. The result is 120
interrupt received AP_READY. Calling fact(6)
interrupt received AP_DONE. The result is 720
interrupt received AP_READY. Calling fact(7)
interrupt received AP_DONE. The result is 5040
interrupt received AP_READY. Calling fact(8)
interrupt received AP_DONE. The result is 40320
interrupt received AP_READY. Calling fact(9)
interrupt received AP_DONE. The result is 362880
interrupt received AP_READY. Calling fact(10)
interrupt received AP_DONE. The result is 3628800
interrupt received AP_READY. Calling fact(11)
interrupt received AP_DONE. The result is 39916800
interrupt received AP_READY. Calli

interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(112)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(113)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(114)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(115)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(116)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(117)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(118)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(119)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(120)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(121)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(122)

interrupt received AP_READY. Calling fact(226)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(227)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(228)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(229)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(230)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(231)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(232)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(233)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(234)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(235)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(236)
interrupt received AP_DONE. The result is 0

interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(367)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(368)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(369)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(370)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(371)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(372)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(373)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(374)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(375)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(376)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(377)

interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(509)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(510)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(511)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(512)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(513)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(514)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(515)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(516)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(517)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(518)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(519)

interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(651)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(652)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(653)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(654)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(655)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(656)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(657)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(658)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(659)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(660)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(661)

interrupt received AP_READY. Calling fact(797)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(798)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(799)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(800)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(801)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(802)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(803)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(804)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(805)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(806)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(807)
interrupt received AP_DONE. The result is 0

interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(937)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(938)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(939)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(940)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(941)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(942)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(943)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(944)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(945)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(946)
interrupt received AP_DONE. The result is 0
interrupt received AP_READY. Calling fact(947)

NameError: name 'ip_task' is not defined

CPU utilization returns percentage used on each core of the ARM

### Run factorial on the ARM and see utilization

In [4]:
import time  
from datetime import datetime
import math
from psutil import cpu_percent

cpu_percent(percpu=True)
for i in range(1000):
    math.factorial(i)
cpu_used = cpu_percent(percpu=True)
# Printing the CPU utilization
print('CPU Utilization = {cpu_used}'.format(**locals()))  

CPU Utilization = [5.7, 100.0]
